# Compuer Science Project

This project has many purpouses, so it's quite hard to sum them all up.
In general, the project is aimed to manage a main file containing information about buses route and a collection with a csv file for each bus route containing detailed information aboute the trip.

It's important to keep in mind, that due to the huge amount of data, the whole Jupyter Notebook takes at laest one hour to run entirely.

Creation of the Pandas Dataframe, call for all the libriaries needed to achive the goals of the project. I transfer all the datasets in a folder called 'Data', located in the same folder of the file .ipynb.

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import pyodbc
import os
import fnmatch
import glob
import re

In [2]:
#this shell has to be run once, then don't run it again ot it will raise an error
dir_progetto=os.getcwd()
dir_data = dir_progetto + '/Data/'
dir_sample=dir_progetto +'/sample/'
dir_metadata = dir_progetto+'/metadata/'
os.chdir(dir_metadata)

In [3]:
#useful later, to enter some parameters in the function 'read_column(column)' and to do some for cycles.
folder_path = dir_data
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
chunk_size = 1

In [4]:
def read_column(column):
    '''This function takes in input a column of the detailed dataset 
    (stored in the folder 'data') and returns a pandas DataFrame 
    with the data of the given column taken from all the csv files. 
    '''
    os.chdir(dir_data)
    total =[]
    for i in range(0, len(csv_files), chunk_size):
        batch_files = csv_files[i:i + chunk_size]
        batch_dfs = []
        for file in batch_files:
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            df=(df[column])
            batch_dfs.append(df)
        batch_df=pd.concat(batch_dfs, ignore_index = True)
        total.append(batch_df)
    total_batch = pd.concat(total, ignore_index=True)
    return(total_batch)


def time_count(colname):
    '''This function takes in input a column from the detaild dataset and
    returns a vector with the total number of seconds (each csv file is represented by a
    value of the vector). The time is measured in this way because every line of the
    csv files is a second.
    '''
    os.chdir(dir_data)
    time_difference = []
    for file in csv_files:
        file_read=pd.read_csv(file)
        total_time = pd.DataFrame(file_read[colname])
        total_time = total_time[total_time[colname]==1]
        if total_time.empty == False:
            time_sum = total_time[colname].count()
            time_difference.append(time_sum)
        else:
            time_sum = 0
            time_difference.append(time_sum)
    return(time_difference)

In [5]:
metadata = pd.read_csv('metaData.csv')
metadata.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


## 1. Extract all trips with Bus Route 83

In [6]:
metadata[metadata['busRoute']=='83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


## 2. Extract all the *BusRoutes* without number

In [7]:
metadata[metadata['busRoute']=='-']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.1500,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.1500,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.1500,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.1500,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.1500,299.15
25,B183_2019-06-01_05-05-41_2019-06-01_15-15-16,183,2019-06-01T05:05:41Z,1559365541,2019-06-01T15:15:16Z,1559402116,132710.70,-,8.370118e+08,6.709220,0,21,0.857666,296.6994,288.1500,303.15
26,B183_2019-06-01_15-19-38_2019-06-01_23-12-03,183,2019-06-01T15:19:38Z,1559402378,2019-06-01T23:12:03Z,1559430723,104331.10,-,5.065605e+08,6.402878,0,27,0.730227,297.3314,292.1500,302.15
34,B183_2019-06-21_02-54-48_2019-06-21_22-37-37,183,2019-06-21T02:54:48Z,1561085688,2019-06-21T22:37:37Z,1561156657,256945.60,-,1.603927e+09,7.909810,0,35,0.766380,294.5689,288.4197,299.15
36,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,183,2019-06-24T03:16:13Z,1561346173,2019-06-24T18:54:06Z,1561402446,205184.90,-,1.462189e+09,9.739464,0,39,0.783150,300.0062,291.5750,306.15
37,B183_2019-06-25_03-22-22_2019-06-25_22-24-45,183,2019-06-25T03:22:22Z,1561432942,2019-06-25T22:24:45Z,1561501485,243475.20,-,1.733085e+09,9.943307,0,49,0.804870,301.8046,293.1500,308.15


## 3. For each (*busNumber*, *busRoute*) determine the number of trips

In [8]:
trips= metadata.groupby(['busNumber', 'busRoute'])['name']
trips.count()

busNumber  busRoute
183        -            11
           31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
Name: name, dtype: int64

## 4. Ratio between the energy consumption and the avarage number of passengers

In [9]:
ratio = metadata['energyConsumption']/metadata['itcs_numberOfPassengers_mean']
print(ratio)

0       8.640500e+07
1       1.214747e+07
2       7.342794e+07
3       1.673328e+08
4       2.613190e+07
            ...     
1404    1.070215e+07
1405    5.338462e+07
1406    4.738047e+07
1407    2.247753e+07
1408    3.345769e+07
Length: 1409, dtype: float64


## 5. For each station, determine the mean number of passengeres

In [10]:
#creating the overall dataset, reading and combining all the csv files
os.chdir(dir_data)
#csv_files = glob.glob(path + "/*.csv")
dfs= []
for i in range(0, len(csv_files), chunk_size):
    batch_files = csv_files[i:i+chunk_size]
    batch_dfs = [pd.read_csv(file) for file in batch_files]
    batch_big_df = pd.concat(batch_dfs, ignore_index=True)
    big_dfstop = batch_big_df[batch_big_df['itcs_stopName'] != '-']
    dfs.append(big_dfstop)

# Concatenate all batches into one DataFrame
big_df = pd.concat(dfs, ignore_index=True)

In [11]:
#Associating new variables to the old columns in the overall df
busstop_name= big_df['itcs_stopName'].drop_duplicates()
pass_mean= big_df.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()

In [12]:
#This creates a new table with only the usful columns
final=pd.merge(busstop_name, pass_mean, on='itcs_stopName')
final

,itcs_stopName,itcs_numberOfPassengers
0,"Zürich, Herdernstrasse",8.332344
1,"Zürich, SBB-Werkstätte",9.406940
2,"Zürich, Letzipark",9.559411
3,"Zürich, Letzibach",9.692340
4,"Zürich, Bahnhof Altstetten",7.816580
...,...,...
144,"Zürich, Farbhof",8.467949
145,"Zürich, Micafil",3.551020
146,"Zürich, Hermetschloo",1.513514
147,"Zürich, Waidhof",0.464286


## 6. For each station, determine the buses that have stopped there at least once

In [53]:
stop=big_df.groupby('itcs_stopName')['itcs_busRoute'].count()
stop


itcs_stopName
Zürich, Albisrank                 12523
Zürich, Albisriederplatz          18851
Zürich, Altes Krematorium         17199
Zürich, Bahnhof Affoltern            67
Zürich, Bahnhof Altstetten         7322
                                  ...  
Zürich, Zentrum Witikon             196
Zürich, Zweiackerstrasse            227
Zürich, Zwielplatz                   81
Zürich, Zwinglihaus                4022
Zürich,Kalkbreite/Bhf.Wiedikon      535
Name: itcs_busRoute, Length: 149, dtype: int64

## 7. For each station, determine the buses that have stopped there at least ten times

In [61]:
stop=big_df.groupby('itcs_stopName')['itcs_busRoute'].count() #this set a limit to at least 10 counts.
print(stop[stop >10])


itcs_stopName
Zürich, Albisrank                 12523
Zürich, Albisriederplatz          18851
Zürich, Altes Krematorium         17199
Zürich, Bahnhof Altstetten         7322
Zürich, Bahnhof Hardbrücke        19270
                                  ...  
Zürich, Zehntenhausplatz            538
Zürich, Zentrum Witikon             196
Zürich, Zweiackerstrasse            227
Zürich, Zwinglihaus                4022
Zürich,Kalkbreite/Bhf.Wiedikon      535
Name: itcs_busRoute, Length: 133, dtype: int64


The results doesn't change because the counts are higher than 10 in all the cases.

## 9. Rate between energy consumption and total distance

In [14]:
os.chdir(dir_metadata)
schema=metadata.groupby(['busNumber', 'busRoute'])
total_distance=metadata.groupby(['busNumber', 'busRoute'])['drivenDistance'].sum()
total_energy = metadata.groupby(['busNumber', 'busRoute'])['energyConsumption'].sum()
ratio= total_energy/total_distance
ratio

busNumber  busRoute
183        -           6230.233181
           31          6086.891785
           32          6174.585331
           33          5970.747507
           46          5631.035959
           72          5899.029643
           83          5844.977154
           N1          5980.920966
           N2          5701.850332
           N4          6154.986937
208        31          5290.862713
           32          5491.247671
           33          5639.244537
           46          5583.110707
           72          5410.318878
           83          5819.638891
           N1          5640.061883
           N2          5405.152963
           N4          6067.190112
dtype: float64

The results are expressed in J/m

## 10. Max and min energy consumption

In [15]:
min_energy_cons= metadata.groupby(['busNumber', 'busRoute'])['energyConsumption'].min()
max_energy_cons = metadata.groupby(['busNumber', 'busRoute'])['energyConsumption'].max()
min_max_en= pd.merge(min_energy_cons, max_energy_cons, on=['busNumber','busRoute'], suffixes=['_min', '_max'])
min_max_en

energyConsumption_min  energyConsumption_max
busNumber busRoute                                              
183       -                   281986700.0           1.733085e+09
          31                  402258500.0           1.907228e+09
          32                  334372500.0           2.477140e+09
          33                  289640300.0           2.265177e+09
          46                  234072800.0           2.684731e+09
          72                  209178600.0           2.159096e+09
          83                  194887200.0           1.692171e+09
          N1                  318349500.0           5.251683e+08
          N2                  240702000.0           5.321266e+08
          N4                  291898900.0           5.486964e+08
208       31                  529832400.0           1.525641e+09
          32                  247156300.0           2.060297e+09
          33                  277535000.0           2.370686e+09
          46                  341292800.0           2.338496e+09
          72                  246544700.0           1.696576e+09
          83                  194741200.0           1.662635e+09
          N1                  281885100.0           4.519165e+08
          N2                  276740700.0           5.350956e+08
          N4                  365690800.0           5.548334e+08

Take into account that the energy consuption minimum is of the order of $10^8$, so there is an order of magnitude in differce between the minum and maximum energy consuption.

## 11. Which bus maximises the difference in energy consumption

In [16]:
min_max_en['difference']= min_max_en['energyConsumption_max']-min_max_en['energyConsumption_min']
print(min_max_en['difference'].sort_values(ascending=False).head(1))

busNumber  busRoute
183        46          2.450658e+09
Name: difference, dtype: float64


## 12. *gnss_altitude* extracted from details and altitude difference

In [17]:
altitude = read_column('gnss_altitude')
big_dfalt = pd.DataFrame(altitude)
big_dfalt = big_dfalt[big_dfalt['gnss_altitude'].diff() != 0]
big_dfalt['diff_altitude']= big_dfalt.diff() #creating new column
print(big_dfalt)

          gnss_altitude  diff_altitude
0              413.6834            NaN
1              414.1855         0.5021
2              414.6841         0.4986
3              415.2814         0.5973
4              415.8791         0.5977
...                 ...            ...
48674457       396.2567        -0.2730
48674458       396.0050        -0.2517
48674459       397.0198         1.0148
48674460       398.0605         1.0407
48674461       398.1967         0.1362

[42101814 rows x 2 columns]


## 13. Total altitude variation for each bus trip

In [18]:
#In this case, the only thing to do was reading all the files and for each of them allocate the total altitude
total_altitude = []
altitude_var=[]
for file in csv_files:
    file_read=pd.read_csv(file)
    file_read = pd.DataFrame(file_read)
    altitude = file_read['gnss_altitude']
    altitude_var = altitude.diff().abs()
    total = altitude_var.sum()
    total_altitude.append(total)
    altitude_var = []
    
i=0
for file in csv_files:
    print('The total altitude for ', file, 'is ',np.round(total_altitude[i],decimals=1))
    i+=1

The total altitude for  B208_2022-06-21_03-55-04_2022-06-21_12-43-14.csv is  5625.2
The total altitude for  B183_2021-12-29_14-22-59_2021-12-29_19-34-35.csv is  8249.9
The total altitude for  B208_2022-04-22_22-35-56_2022-04-23_02-24-18.csv is  3256.3
The total altitude for  B183_2020-05-15_13-32-53_2020-05-15_18-35-50.csv is  6216.8
The total altitude for  B208_2021-12-08_05-09-40_2021-12-08_19-18-00.csv is  9109.2
The total altitude for  B183_2019-11-24_04-08-51_2019-11-24_22-11-57.csv is  27499.7
The total altitude for  B183_2022-01-08_23-40-17_2022-01-09_03-35-32.csv is  5238.4
The total altitude for  B183_2019-12-01_16-50-02_2019-12-01_23-30-17.csv is  10684.9
The total altitude for  B183_2022-09-05_04-16-26_2022-09-05_18-07-32.csv is  19343.7
The total altitude for  B183_2022-02-26_23-34-31_2022-02-27_03-36-49.csv is  5782.1
The total altitude for  B183_2021-06-01_13-31-06_2021-06-01_18-33-34.csv is  6303.6
The total altitude for  B208_2021-12-07_05-12-17_2021-12-07_19-02-13.csv 

The total altitude is expressed in meters, as the data referres to Zurich's buses. The city is placed apon montains/hills, so the results are reasonable

## 14. Mean ambient temperature for each month

In [19]:
#Reading the column 'time_unix' and storing the data processed into big_df_time
unixtime=read_column('time_unix')
to_date=[]
to_date.append(pd.to_datetime(unixtime.astype(int), unit='s',origin='unix'))    #convert unix in date
big_df_time = pd.concat(to_date, ignore_index=True)
big_df_time =pd.DataFrame(big_df_time)
big_df_time = big_df_time.rename(columns={'time_unix':'datetime'})
big_df_time['month'] = big_df_time['datetime'].dt.month     #extracts the months from the dates

In [20]:
#Same with the column 'temperature_ambient'
temperature=read_column('temperature_ambient')
temperature=pd.DataFrame(temperature)
final_df=pd.concat([big_df_time, temperature], axis=1)
mean_temperature=final_df.groupby('month')['temperature_ambient'].mean() #computing the mean of temperature for
                                                                         #each month
mean_temperature

month
1     278.801440
2     281.205427
3     283.231337
4     287.037925
5     290.178378
6     296.771650
7     297.716663
8     296.327105
9     292.442343
10    287.894845
11    281.904302
12    279.562856
Name: temperature_ambient, dtype: float64

The average month temperature is lower in the winter months (1,2,11,12) and hotter in the summer months, as we espected. The conversion Kelvin - Celsius degrees brings the results to be more readable, in this case, in december the mean temperature is 6.5 degree, while in august the mean temperature is 23.5 degree.

## 15. Total halt time and total park time and ratio

In [21]:
#calling the function time_count(colname)
time_difference_halt = time_count('status_haltBrakeIsActive')
time_difference_park = time_count('status_parkBrakeIsActive')

For the cell below: the first column of the DataFrame is the name (number) of the bus, the second column is the total halt time for each bus line (expressed in seconds), the third column is the total park time for each bus line (expressed in seconds) and the fourth column is the ration between the second and the third column.

In [22]:
os.chdir(dir_metadata)
busnumber = metadata['busRoute']
name = metadata['name']
file_list=[]
for file in csv_files:
    file_list.append(file)
file_list = pd.DataFrame(file_list, columns=['name'])
new_list = pd.merge(file_list, busnumber, on=name)
time_difference_halt = pd.DataFrame(time_difference_halt)
time_difference_park = pd.DataFrame(time_difference_park)
ratio = time_difference_halt/time_difference_park
comb = pd.DataFrame(pd.concat([new_list['busRoute'], time_difference_halt, time_difference_park, ratio], axis=1))
total_bus=comb.groupby('busRoute')[0].sum()
total_bus = pd.DataFrame(total_bus)
total_bus

,0,0,0
busRoute,,,
-,100436,29964,48.614288
31,109828,47455,47.622915
32,274986,91235,84.730994
33,1591186,533742,718.980430
46,1175212,488166,404.052315
72,1526024,545089,710.283656
83,8231512,2988343,3511.681889
N1,152595,68684,136.991901
N2,419255,131768,198.667016


Keep in mind that 1 month corrisponds to $2.5 \cdot 10^{6}$ seconds

## 16. For each consecutive stops, compute the average speed

In [24]:
os.chdir(dir_data)
for file in csv_files:
    file_read=pd.read_csv(file)
    data = pd.DataFrame(file_read[['itcs_stopName', 'odometry_vehicleSpeed']]) #selecting this two columns
    stop = pd.DataFrame(data[data['itcs_stopName'] != '-']) #filtering for rows in 'itcs_stopName'
    mean_vel =[]
    i = 0
    for index, row in data.iterrows():
        if row['itcs_stopName'] != '-':                     #selecting rows with 'itcs_stopName' different than '-'
            vel = data['odometry_vehicleSpeed'][i:index]    #filtering the velocities between one stop and the next one
            mean_vel.append(vel.mean())
            i = index                           #Update the count
    stop['mean_vel'] = mean_vel
    stop = stop.drop(columns = ['odometry_vehicleSpeed'])
    print(stop)


                    itcs_stopName   mean_vel
234        Zürich, Herdernstrasse   2.311542
302        Zürich, SBB-Werkstätte   7.274698
331             Zürich, Letzipark   8.748696
365             Zürich, Letzibach  10.892210
567    Zürich, Bahnhof Altstetten   2.339214
...                           ...        ...
29390  Zürich, Bahnhof Hardbrücke   4.297247
29448           Zürich, Schiffbau   7.074478
29504   Zürich, Escher-Wyss-Platz   6.289818
29577  Zürich, Rosengartenstrasse   6.441937
29963        Zürich, Bucheggplatz   2.073348

[268 rows x 2 columns]
                    itcs_stopName  mean_vel
744        Zürich, Herdernstrasse  0.763022
824        Zürich, SBB-Werkstätte  6.091877
878             Zürich, Letzipark  4.072825
944             Zürich, Letzibach  5.662486
1114   Zürich, Bahnhof Altstetten  2.853360
...                           ...       ...
17731           Zürich, Albisrank  5.419846
17787         Zürich, Flurstrasse  5.667908
17850             Zürich, Kappeli  6.161

This last part of code is really time consuming, that's because I decided to print every result for each file.